# Code to pre-process the .mp4 videos from the Drive&Act Dataset by extracting frames into .npy files with their labels. 

In [1]:
import os
import subprocess
import numpy as np
import cv2
import pandas as pd
import csv

## Define a one-hot-encoding helper function here:

In [2]:
# Might need to modify input "data_set"
def label_encoding(activity):
    if activity == 'sitting_still':
        activity = 0
    elif activity == 'standing_by_the_door':
        activity = 1
    elif activity == 'entering_car':
        activity = 2
    elif activity == 'closing_door_outside':
        activity = 3
    elif activity == 'closing_door_inside':
        activity = 4
    elif activity == 'opening_laptop':
        activity = 5
    elif activity == 'opening_door_inside':
        activity = 6
    elif activity == 'fetching_an_object':
        activity = 7
    elif activity == 'pressing_automation_button':
        activity = 8
    elif activity == 'fastening_seat_belt':
        activity = 9
    elif activity == 'using_multimedia_display':
        activity = 10
    elif activity == 'closing_laptop':
        activity = 11
    elif activity == 'placing_an_object':
        activity = 12
    elif activity == 'interacting_with_phone':
        activity = 13
    elif activity == 'drinking':
        activity = 14
    elif activity == 'opening_bottle':
        activity = 15
    elif activity == 'eating':
        activity = 16
    elif activity == 'preparing_food':
        activity = 17
    elif activity == 'looking_back_left_shoulder':
        activity = 18
    elif activity == 'reading_magazine':
        activity = 19
    elif activity == 'talking_on_phone':
        activity = 20
    elif activity == 'looking_or_moving_around (e.g. searching)':
        activity = 21
    elif activity == 'closing_bottle':
        activity = 22
    elif activity == 'putting_on_jacket':
        activity = 23
    elif activity == 'exiting_car':
        activity = 24
    elif activity == 'opening_door_outside':
        activity = 25
    elif activity == 'writing':
        activity = 26
    elif activity == 'working_on_laptop':
        activity = 27
    elif activity == 'putting_on_sunglasses':
        activity = 28
    elif activity == 'taking_off_sunglasses':
        activity = 29
    elif activity == 'opening_backpack':
        activity = 30
    elif activity == 'taking_laptop_from_backpack':
        activity = 31
    elif activity == 'reading_newspaper':
        activity = 32
    elif activity == 'taking_off_jacket':
        activity = 33
    elif activity == 'unfastening_seat_belt':
        activity = 34
    elif activity == 'putting_laptop_into_backpack':
        activity = 35
    else:
        print(activity)

    return int(activity)

## Test getting frames from single video for single row (activity slice) from the provided .csv file: 

In [3]:
path = "/home/pjsimmon/ECE285_Final_Dataset/activities_3s/kinect_ir/"
root_dir = "/home/pjsimmon/ECE285_Final_Dataset/kinect_ir/"

filename = "midlevel.chunks_90.split_0.train.csv"

In [4]:
!pwd

/home/pjsimmon/ECE285_Final_Dataset


In [5]:
os.path.exists(path + filename)

True

In [6]:
annotations_df = pd.read_csv(path + filename)
annotations_df.head()

,participant_id,file_id,annotation_id,frame_start,frame_end,activity,chunk_id
0,1,vp1/run1b_2018-05-29-14-02-47.kinect_ir,1,116,164,closing_door_outside,0
1,1,vp1/run1b_2018-05-29-14-02-47.kinect_ir,3,204,260,opening_door_outside,0
2,1,vp1/run1b_2018-05-29-14-02-47.kinect_ir,4,260,313,entering_car,0
3,1,vp1/run1b_2018-05-29-14-02-47.kinect_ir,5,313,348,closing_door_inside,0
4,1,vp1/run1b_2018-05-29-14-02-47.kinect_ir,6,348,439,fastening_seat_belt,0


In [ ]:
# Loop through all rows: 
for row in range(0, len(annotations_df)):
    video_filename = str(annotations_df["file_id"][row]) + ".mp4"        # Get video filename for each row

    # Extract the frames using start_frame and end_frame
    start_frame = annotations_df["frame_start"][row]
    end_frame = annotations_df["frame_end"][row]

    activity = annotations_df["activity"][row]
    label = label_encoding(activity)
    
    video_path = root_dir + video_filename
    proc_dir = root_dir

    subprocess.call(["rm", "-rf", root_dir + "proc"])
    subprocess.call(["mkdir", proc_dir + "proc"])

    ffmpeg = "ffmpeg"

    # Extracts 30 images every second from video and resizes images to 224x224 with aspect ratio 4:3
    subprocess.call([ffmpeg, "-i", video_path, "-r", "30", "-s", "224x224", "-aspect", "4:3", proc_dir + "proc/video_process.mp4"])

    #assuming 30 fps 
    fps = 30.0

    # each are specified in seconds
    start_time = float(start_frame) / float(fps)
    end_time = float(end_frame) / float(fps)
    duration = end_time - start_time

    # cut videos into activity chunk segments
    subprocess.call(["rm", "-rf", proc_dir + "tmp"])
    subprocess.call(["mkdir", proc_dir + "tmp"])
    subprocess.call([ffmpeg, "-i", proc_dir + "proc/video_process.mp4", "-ss", str((start_time)), "-to", str((end_time)), proc_dir + "tmp/video_process_cropped.mp4"])

    # sample the vids into 30 fps
    subprocess.call([ffmpeg, "-i", proc_dir + "tmp/video_process_cropped.mp4", "-r", "30", proc_dir + "tmp/thumb%04d.jpg"])
    
    destination_dir = root_dir 

    start = 1
    end = int(fps * duration)

    # loop through all images, 30 fps for DURATION seconds
    for i in range(start, end):
        image_str = proc_dir + "tmp/thumb" + ('%04d' % i) + ".jpg"
        image = cv2.imread(image_str, cv2.IMREAD_COLOR)

        # Convert to Greyscale
        if (i == 1):
            prev_image = image

        print(i)
        # pretty sure our images are already gray, so not sure if we need these pre-processing steps:
        gray_image_prev = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Run optical flow: 
        '''
        dtvl1 = cv2.createOptFlow_DualTVL1()
        image_flowDTVL1 = dtvl1.calc(gray_image_prev, gray_image, None)    

        # Truncate Pixels between [-20, 20]
        image_flow_truncated = np.clip(image_flowDTVL1, -20, 20)

        # Normalize image into [-1, 1]
        norm_image_flow = cv2.normalize(image_flow_truncated, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        # Crop 224x224 samples

        # reshape
        norm_image_flow_r = norm_image_flow.reshape(1, 224, 224, 2)
        '''

        norm_image_flow_r = gray_image

        # Add to Flow numpy
        if (i == 1):
            flow_video = norm_image_flow_r
        else:
            flow_video = np.concatenate((flow_video, norm_image_flow_r), axis=0)

        prev_image = image

        print("Processed image " + str(i))

    #flow_video_r = flow_video.reshape(start, end, 224, 224, 2)

    # when done, copy .npy files for one 10s video clip, RGB and Flow
    subprocess.call(["mkdir", destination_dir + "video_flow/"])
    destination_flow = destination_dir + "video_flow/" + video_filename[4:] + "_" + str(start_frame) + str(end_frame) + ".npy"
    np.save(destination_flow, flow_video)
    print("Processed video " + video_filename + " " + str(start_frame) + " " + str(end_frame))

    txt_path = root_dir + 'video_flow/' + filename[:-3] + 'txt'

    with open(root_dir + 'video_flow/' + filename[:-3] + 'txt', "a") as txt_file: 
        txt_file.write(video_filename[4:] + "_" + str(start_frame) + str(end_frame) + ".npy" + " " + str(label) + "\n")


1
Processed image 1
2
Processed image 2
3
Processed image 3
4
Processed image 4
5
Processed image 5
6
Processed image 6
7
Processed image 7
8
Processed image 8
9
Processed image 9
10
Processed image 10
11
Processed image 11
12
Processed image 12
13
Processed image 13
14
Processed image 14
15
Processed image 15
16
Processed image 16
17
Processed image 17
18
Processed image 18
19
Processed image 19
20
Processed image 20
21
Processed image 21
22
Processed image 22
23
Processed image 23
24
Processed image 24
25
Processed image 25
26
Processed image 26
27
Processed image 27
28
Processed image 28
29
Processed image 29
30
Processed image 30
31
Processed image 31
32
Processed image 32
33
Processed image 33
34
Processed image 34
35
Processed image 35
36
Processed image 36
37
Processed image 37
38
Processed image 38
39
Processed image 39
40
Processed image 40
41
Processed image 41
42
Processed image 42
43
Processed image 43
44
Processed image 44
45
Processed image 45
46
Processed image 46
47
Pro

In [ ]:
import datetime
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
print("Started running experiment to extract all .npy files for single .CSV at 3:46pm.")